In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from tqdm import tqdm
import seaborn as sns
pd.set_option('display.max_columns',None)
from sklearn.metrics import roc_auc_score
TARGET = 'answered_correctly'

In [2]:
val = pd.read_feather('../features/kernel_base/BASE_FIX_valid.feather')
oof1 = pd.read_feather('../data/oof/061_simple_lgb.feather')
oof2 = pd.read_feather('../data/oof/059_simple_lgb.feather')
oof3 = pd.read_feather('../data/oof/060_simple_lgb.feather')

In [6]:
oof1.columns = ['row_id','pred1']
oof2.columns = ['row_id','pred2']
oof3.columns = ['row_id','pred3']

In [9]:
df = pd.concat([val,oof1['pred1'],oof2['pred2'],oof3['pred3']],axis=1)

In [16]:
df['pred'] = df['pred1'] + df['pred2'] + df['pred3']

In [26]:
df['pred'] = df['pred1']*0.3 + df['pred3'] * 0.7
roc_auc_score(df[TARGET],df['pred'])

0.7847826278582267